In [16]:
import pandas as pd

df = pd.read_pickle('../data/final_data.pkl')

In [17]:
# MODEL PREDICTION 'Code_local_type' == 1, MAISON 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# MSE : 3 184 771 589 078.1714 avec 5 colonnes et 100 000 lignes au début du programme (régression linéaire)
# MSE : 9 351 187 946 604.617 avec 11 colonnes et 700 000 lignes au début du programme
# MSE : 9 621 056 123 391.139 en changeant le code type local == 1
# MSE : 2 642 213 419 979.8486 avec 5 colonnes et 100 000 lignes au début du programme (avec year, code postal, et nature mutation) (code local == 1)
# MSE : 2 642 223 129 457.908 avec pareil mais Lasso (few features should be important) (code local == 1)
# MSE : 2 283 126 024 532.157 avec pareil mais Ridge (lot of features should be important) (code local == 1)
# MSE : 1 481 548 894 581.531 avec Ridge mais 300 000 lignes (code local == 1)
# MSE : 1 419 765 756 499.442 avec Lasso mais 300 000 lignes (code local == 1)
# MSE : 15 011 506 135 748.002 avec Lasso mais 1 021 105 lignes (code local == 1)
# MSE : 6 011 506 135 748.002 avec Ridge mais ... lignes (code local == 1)

df_test = df.copy()
# test seulement avec le code local == 1
df_test = df_test[df_test['Code_type_local'] == 1]
df_test = df_test.drop(columns=["Code_type_local"])


print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere','Commune','day'])
print(X.columns)


X = pd.get_dummies(X, columns=['year'])


y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

# Initialisez le modèle Ridge
modele_ridge_code_1 = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge_code_1.fit(X_train, y_train)
# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge_code_1.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions,squared=False)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


nombre individus dataframe global :  1890477 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'month', 'year', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  85382.85077795523
         Valeur_reelle  Valeur_predite
2913513       250000.0   269968.739448
5807163       382000.0   183472.689157
3503003       345371.0   282463.387758
5873890       385000.0   422432.318913
3904895       282750.0   164334.399042
...                ...             ...
4238101       244000.0   255529.801596
1784502       125000.0   119077.619522
3038752       284970.0   235707.634293
2895212       285000.0   348688.942115
2025959       168000.0   169318.630059

[945239 rows x 2 columns]
R carré : 0.43077278867293534


In [18]:
# MODEL PREDICTION 'Code_local_type' == 2, APPARTEMENT

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 2
df_test = df_test[df_test['Code_type_local'] == 2]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere','Commune','day'])

print(X.columns)

X = pd.get_dummies(X, columns=['year'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge_code_2 = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge_code_2.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge_code_2.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions,squared=False)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


Code_type_local
1    1890477
2    1087851
3     913185
4     193431
Name: count, dtype: int64
nombre individus dataframe global :  1087851 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'month', 'year', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  89503.6814683054
         Valeur_reelle  Valeur_predite
5489647       254100.0   149428.493592
5892616       289000.0   174225.490255
2859891       390000.0   181020.621259
3532971       400000.0   151902.812466
5719692        77000.0   197702.622266
...                ...             ...
6338433       119000.0   157740.169203
3677625       105000.0   227755.621379
5251114       414400.0   286593.218691
1827420       140000.0   157559.867725
469651        290000.0   106206.779678

[217571 rows x 2 columns]
R carré : 0.31002738120819895


In [19]:
# MODEL PREDICTION 'Code_local_type' == 3, DEPENDANCE


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 3
df_test = df_test[df_test['Code_type_local'] == 3]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere','Commune','day'])
print(X.columns)

X = pd.get_dummies(X, columns=['year'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge_code_3 = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge_code_3.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge_code_3.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions,squared=False)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


Code_type_local
1    1890477
2    1087851
3     913185
4     193431
Name: count, dtype: int64
nombre individus dataframe global :  913185 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'month', 'year', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  107800.34755940063
         Valeur_reelle  Valeur_predite
5151380       161100.0   103685.834992
3512305        87000.0   111462.472875
6470235       155000.0   150628.618320
5901950        76500.0   199004.493116
3506959       168000.0   152167.756651
...                ...             ...
3154972       135000.0   154404.649296
2150222        96000.0   139091.975330
2567023        95000.0   114415.295263
881681        152000.0   105378.329625
5274793        30000.0   269086.008439

[182637 rows x 2 columns]
R carré : 0.13381716608951855


In [20]:
# MODEL PREDICTION 'Code_local_type' == 4, local indus


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score


df_test = df.copy()

# test seulement avec le code local == 4
df_test = df_test[df_test['Code_type_local'] == 4]
print(df['Code_type_local'].value_counts())

df_test = df_test.drop(columns=["Code_type_local"])

print("nombre individus dataframe global : ", len(df_test), "\n")


# test drop colonne
X = df_test.drop(columns=['Valeur_fonciere','Commune','day'])
print(X.columns)

X = pd.get_dummies(X, columns=['year'])

y = df_test['Valeur_fonciere']  # Variable cible

# Séparez les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initialisez le modèle Ridge
modele_ridge_code_4 = Ridge(alpha=1.0)  # L'alpha est le paramètre de régularisation

modele_ridge_code_4.fit(X_train, y_train)

# Entraînez le modèle sur les donné
# Prédisez les valeurs sur l'ensemble de test
predictions = modele_ridge_code_4.predict(X_test)

# Calculez l'erreur quadratique moyenne (MSE) pour évaluer la performance du modèle
mse = mean_squared_error(y_test, predictions,squared=False)

# Affichez l'erreur quadratique moyenne
print("Erreur Quadratique Moyenne (MSE) : ", mse)

# Créez un DataFrame avec les valeurs prédites et les valeurs réelles
resultats = pd.DataFrame(
    {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# Affichez les valeurs prédites par rapport aux valeurs réelles
print(resultats)
# Calculez le R carré
r2 = r2_score(y_test, predictions)
# Affichez le R carré
print("R carré :", r2)


Code_type_local
1    1890477
2    1087851
3     913185
4     193431
Name: count, dtype: int64
nombre individus dataframe global :  193431 

Index(['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
       'month', 'year', 'Prix_moyen_m2'],
      dtype='object')
Erreur Quadratique Moyenne (MSE) :  123021.6872232916
         Valeur_reelle  Valeur_predite
3148075       125000.0   165016.512343
4738414       360000.0   219503.272877
389674        151000.0   191138.138009
1041502       196000.0   169238.226423
3078068       200000.0   142883.773706
...                ...             ...
2604077        44000.0   184020.204453
3196100        85000.0   143014.917915
2078909        71284.8   191493.290963
2409322       373000.0   197731.709077
2774694        80000.0   159422.243414

[38687 rows x 2 columns]
R carré : 0.07287105848145126


In [21]:
# Export Model

import pickle as pkl

with open('../model/modele_regression_code_1.pkl','wb') as fichier_pickler:
    pkl.dump(modele_ridge_code_1,fichier_pickler)

with open('../model/modele_regression_code_2.pkl','wb') as fichier_pickler:
    pkl.dump(modele_ridge_code_2,fichier_pickler)

with open('../model/modele_regression_code_3.pkl','wb') as fichier_pickler:
    pkl.dump(modele_ridge_code_3,fichier_pickler)

with open('../model/modele_regression_code_4.pkl','wb') as fichier_pickler:
    pkl.dump(modele_ridge_code_4,fichier_pickler)
